In [1]:
# import libraries 
import pandas as pd
import numpy as np
import re

In [7]:
# read the library csv file generated by peptide_preparation.R
#master = pd.read_csv('/home/david/PL_projects/motif_library/PBD_toolkit/library_04102024.csv', 
 #                    delimiter=',')

master = pd.read_csv('/home/david/PL_projects/PL_papers/Scaffold_Letters/Code/Scaffold_PCA/PBD_selection/peptide_selection.csv')

# Retrieve the sequences of interest : 'peptide_sequence' or 'Domain_Sequence'
peptide_seq = master['Domain_Sequence'].unique()
peptide_seq = np.array(peptide_seq)

#domain_seq = master['Domain_Sequence'].unique()
#domain_seq = np.array(domain_seq)

In [8]:
# Import the reference data for yeast codon optimization
aa = pd.read_csv('/home/david/PL_projects/PL_papers/Scaffold_Letters/Code/Scaffold_PCA/PBD_selection/reference_data/dict_codon.csv', 
                 header = 0, names = ['codon', 'aa'])
usage = pd.read_csv('/home/david/PL_projects/PL_papers/Scaffold_Letters/Code/Scaffold_PCA/PBD_selection/reference_data/dict_usage.csv', 
                    header = 0, names = ['codon', 'usage'])
# merge both df
usage = aa.merge(usage, on = 'codon')


In [4]:
# Function performing the codon optimization
## Input
# seq_array :  array of the aa sequence to retro-translate to codon
# usage : pd dataframe of 3 columns. codon :  dna sequence of codons, aa : encoded aa of each codon, 
#         usage : the frequencies of usage of each codon. If only one codon encodes for an aa, usage = 1
# aa_diversity : aa for which you want to degenerate its codon, default = 'X'
# codon_diversity : codon used to encode the aa to degenerate (aa_diversity)
# nb_diversity : number of aa (aa_diversity) to degenerate, default = 2
# site :  dna sequence of a restriction site to avoid in the encoding, if site is found in the sequence 
#         in progress of encoding, it will replace the last encoded codon by the second used codon if possible, 
#         if not, a warning will be raise in the output. Default:'AAGCTT', HindIII site

## Ouput
# A pd dataframe with 4 columns. 
    # aa_seq : aa sequence that was retrotranscribed
    # dna_seq : optimized codon sequence of aa_seq
    # enz_warning : if the restriction site is present in the dna_seq
    #
def aa_to_dna(seq_array, usage, aa_diversity = 'X', 
              codon_diversity = 'NNK', nb_diversity = 2, site = 'AAGCTT'): 
    # define output object
    out = pd.DataFrame(columns = ('aa_seq', 'dna_seq', 'enz_warning', 'correct_len'))
    
    # find codon most used for each aa in yeast
    higher_usage = usage[usage.groupby(['aa'])['usage'].transform(max) == usage['usage']]

    # loop through aa sequences of the input
    for seq in seq_array:
    #divide aa sequence in single character
        seq_split = [*seq]
        # set dna sequence object
        dna_seq = ('')
        # define value for diversity aa
        j = 0
        # define object to save the previous codon of each aa
        codon_prev=''
        # define object to save if the restriction site not avoidable from the encoding
        warn =''
        # loop to assign a codon to each aa
        for aa in seq_split:
            # associate each aa to a codon
            possible_codes = usage[usage['aa']== aa]
            # sort with highest usage on top
            sorted_codes = possible_codes.sort_values(by='usage', ascending=False)
            code = sorted_codes['codon'].iloc[0]
            
            # add diversity to the sequences by changing a codon of a specific aa
            # by a codon with a degenerate position for the number of positions 
            # equal to nb_diversity
            if (aa == aa_diversity) & (j < nb_diversity):
                j = j+1
                code = codon_diversity
                codon_prev=code
            
            # If there is only one codon encoding for an aa
            # keep the same codon even when one after the other 
            elif(possible_codes['codon'].nunique() < 2):
                codon = higher_usage[higher_usage['aa'] == aa]
                code = codon['codon'].iloc[0]
            # If one aa is repeated, alternate the codon between the first and 
            # second highest used codon
            elif((codon_prev==code) & (possible_codes['codon'].nunique()>=2)):
                code = sorted_codes['codon'].iloc[1]
                
            
            # verify for a restriction site in the dna sequence
            t = dna_seq+code
            
            if(re.search(site ,t)):
                # assemble all possible sequence for this aa
                all_code =  dna_seq + sorted_codes['codon']
                
               # check which one have the resriction site
                l = pd.Series(all_code).str.contains(site)
                # if only one codon possible, keep the codon but add a warning
                if(len(sorted_codes.loc[~l]) == 0):
                    info = sorted_codes.loc[l].iloc[0]
                    warn = site

                # if there is one possibility to avoid de restriction site, 
                # use it even if it is not optimized
                elif(len(sorted_codes.loc[~l]) > 0):
                    info = sorted_codes.loc[~l].iloc[0]
                    code =info['codon']                  
            
            # concatenate the dna sequence with the new codon 
            dna_seq = dna_seq + code
            codon_prev = code
        # generate a warning if the dna length is not 3*aa sequence lenght
        warn_len = len(dna_seq) == len(seq)*3

        # build the output dataframe for one sequence
        one = pd.DataFrame({'aa_seq' : [seq], 
                            'dna_seq' : [dna_seq], 
                            'enz_warning' : [warn], 
                            'correct_len' : [warn_len]})
        
        # concatenate the one df with the previous ones
        out = pd.concat([out, one], ignore_index = True, axis = 0)
    return(out)  
    

In [10]:
# Perform codon optimization for the sequences of interest
peptide_trans = aa_to_dna(peptide_seq, usage, site ='AAGCTT')
#domain_trans = aa_to_dna(peptide_seq, usage, site ='AAGCTT')

/tmp/ipykernel_3871892/1985150886.py:25: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  higher_usage = usage[usage.groupby(['aa'])['usage'].transform(max) == usage['usage']]


In [11]:
peptide_trans.to_csv('/home/david/PL_projects/PL_papers/Scaffold_Letters/Code/Scaffold_PCA/PBD_selection/peptide_DNA.csv')
domain_trans.to_csv('/home/david/PL_projects/PL_papers/Scaffold_Letters/Code/Scaffold_PCA/PBD_selection/domain_DNA.csv')
